In [1]:
import flopy
import pandas as pd
import numpy as np

In [2]:
top_raw = pd.read_csv(r'C:\Users\Annie\Desktop\UIUC\Spring 2022\Hydro w python\top_elevation.csv')
bottoms_raw = pd.read_csv(r'C:\Users\Annie\Desktop\UIUC\Spring 2022\Hydro w python\bottom_elevations.csv')

In [73]:
# define necessary parameters
nlay = 10
nrow = top_raw['row'].max()
ncol = top_raw['column'].max()

In [64]:
# create 2D array of floats for top elevations with shape (nrow,ncol)

# create empty arrays with proper shape that will be populated with elevation data
top_elevation = np.zeros(shape=(top_raw['row'].max(),top_raw['column'].max()))
bottom_elevations = np.zeros(shape=(nlay,bottoms_raw['row'].max(),bottoms_raw['column'].max()))

# loop through raw data to populate empty arrays with elevation data
# top
for i in range(len(top_raw['FID'])):
    top_elevation[int(top_raw.iloc[[i]]['row'])-1][int(top_raw.iloc[[i]]['column'])-1] = float(top_raw.iloc[[i]]['top'])

In [67]:
# looping through elevation data to populate bottom layer arrays
for layer in range(nlay):
    for i in range(len(bottoms_raw['FID'])):
        bottom_elevations[layer][int(bottoms_raw.iloc[[i]]['row'])-1][int(bottoms_raw.iloc[[i]]['column'])-1] = float(bottoms_raw.iloc[[i]]['Bottom'+str(layer+1)])

In [69]:
print(top_elevation)
bottom_elevations

[[980. 968. 978. ... 866. 838. 854.]
 [976. 967. 977. ... 907. 898. 936.]
 [973. 964. 967. ... 921. 901. 903.]
 ...
 [844. 847. 856. ... 784. 792. 821.]
 [852. 854. 859. ... 776. 782. 837.]
 [853. 854. 857. ... 783. 809. 840.]]


array([[[979. , 967. , 977. , ..., 865. , 837. , 853. ],
        [975. , 966. , 976. , ..., 906. , 897. , 935. ],
        [972. , 963. , 966. , ..., 920. , 900. , 902. ],
        ...,
        [843. , 846. , 855. , ..., 783. , 791. , 820. ],
        [851. , 853. , 858. , ..., 775. , 781. , 836. ],
        [852. , 853. , 856. , ..., 782. , 808. , 839. ]],

       [[978. , 966. , 976. , ..., 738. , 728. , 717. ],
        [974. , 965. , 975. , ..., 747. , 736. , 725. ],
        [971. , 962. , 965. , ..., 753.1, 741. , 730. ],
        ...,
        [842. , 845. , 854. , ..., 750. , 746. , 742. ],
        [850. , 852. , 857. , ..., 753. , 748. , 744. ],
        [851. , 852. , 855. , ..., 754. , 749. , 745. ]],

       [[977. , 965. , 975. , ..., 737. , 727. , 716. ],
        [973. , 964. , 974. , ..., 746. , 735. , 724. ],
        [970. , 961. , 964. , ..., 752.1, 740. , 729. ],
        ...,
        [841. , 844. , 853. , ..., 709. , 704. , 699. ],
        [849. , 851. , 856. , ..., 707. , 701

In [71]:
# change units to meters from feet
top_elevation_m = 0.3048*top_elevation
bottom_elevations_m = 0.3048*bottom_elevations

In [74]:
# create model object
modelname = "my_pumping_model"
m = flopy.modflow.Modflow(modelname, exe_name = 'mf2005')

# create flopy discretization object, length and time are meters (2) and days (4)
dis = flopy.modflow.ModflowDis(model=m, nlay=nlay, nrow=nrow, ncol=ncol, top=top_elevation_m, botm=bottom_elevations_m, 
                               itmuni = 4, lenuni = 2)